In [1]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd

In [2]:
def get_properties(page=1):    
    res = requests.get(f'https://www.rightmove.co.uk/house-prices/sw11-1lu.html?page={page}')
    soup = BeautifulSoup(res.text, 'html.parser')
    s = soup.find_all('script')[-2]
    d = json.loads(s.string[s.string.find('{'):])
    df = pd.DataFrame(d['results']['properties'])
    df = df.drop('transactions', axis=1).join(df.transactions.explode()).reset_index(drop=True)
    df = df.drop('transactions', axis=1).join(pd.json_normalize(df.transactions))
    return df

In [3]:
df = pd.concat([get_properties(page+1) for page in range(1)]).reset_index(drop=True)

In [4]:
df['numericPrice'] = pd.to_numeric(df.displayPrice.str.replace('£', '').str.replace(',', ''))

In [5]:
df_address = df.address.str.split(', ').explode().to_frame()

In [6]:
postcode_regex = r'[A-Z]{1,2}[0-9]{1,2}[ ]{0,1}[0-9]{1,2}[A-Z]{1,2}$'
df_address['postcode'] = df_address.address.str.extract(f'({postcode_regex})')
df_address.address = df_address.apply(lambda x: x.address if isinstance(x.postcode, float) else x.address.replace(x.postcode, ''), axis=1)

In [7]:
df_address['number'] = pd.to_numeric(df_address.address.str.replace(r'[^0-9]*$', '', regex=True), errors='coerce')
df_address['number'] = df_address.apply(lambda x: x.address if x.number > 0 else None, axis=1)

In [8]:
df_address['prefix'] = df_address.number.notna().groupby(level=0).cumsum() == 0
df_address['prefix'] = df_address.apply(lambda x: x.address if x.prefix else None, axis=1)

In [9]:
df_address.loc[df_address.notna().sum(axis=1) > 1, 'address'] = None

In [10]:
address_to_idx = [None] + df_address.address.dropna().drop_duplicates().tolist()
df_address['counter'] = df_address.address.apply(lambda x: address_to_idx.index(x)).replace({0: None})
df_address['counter'] = df_address.counter.groupby(level=0).rank(method='first').convert_dtypes()

In [11]:
df_address_melt = df_address.reset_index().melt(id_vars=['index', 'counter']).dropna(subset='value')

In [12]:
df_address_melt['column'] = (df_address_melt.variable + '_' + df_address_melt.counter.astype(str)).str.replace('_<NA>', '')

In [13]:
df_address = df_address_melt.pivot(index='index', columns='column', values='value')

In [14]:
df = df.join(df_address)

In [15]:
df.to_csv('parma_crescent_prices.csv', index=False)

In [16]:
df

,address,propertyType,bedrooms,images,hasFloorPlan,location,detailUrl,displayPrice,dateSold,tenure,newBuild,numericPrice,address_1,address_2,number,postcode,prefix
0,"76, Parma Crescent, London, Greater London SW1...",Terraced,5.0,{'imageUrl': 'https://media.rightmove.co.uk/di...,True,"{'lat': 51.4622, 'lng': -0.16433}",https://www.rightmove.co.uk/house-prices/detai...,"£2,450,000",19 Oct 2023,Freehold,False,2450000,Parma Crescent,London,76,SW11 1LU,NaN
1,"76, Parma Crescent, London, Greater London SW1...",Terraced,5.0,{'imageUrl': 'https://media.rightmove.co.uk/di...,True,"{'lat': 51.4622, 'lng': -0.16433}",https://www.rightmove.co.uk/house-prices/detai...,"£1,425,000",22 Apr 2016,Freehold,False,1425000,Parma Crescent,London,76,SW11 1LU,NaN
2,"76, Parma Crescent, London, Greater London SW1...",Terraced,5.0,{'imageUrl': 'https://media.rightmove.co.uk/di...,True,"{'lat': 51.4622, 'lng': -0.16433}",https://www.rightmove.co.uk/house-prices/detai...,"£690,000",17 Feb 2006,Freehold,False,690000,Parma Crescent,London,76,SW11 1LU,NaN
3,"76, Parma Crescent, London, Greater London SW1...",Terraced,5.0,{'imageUrl': 'https://media.rightmove.co.uk/di...,True,"{'lat': 51.4622, 'lng': -0.16433}",https://www.rightmove.co.uk/house-prices/detai...,"£150,000",28 Feb 2002,Freehold,False,150000,Parma Crescent,London,76,SW11 1LU,NaN
4,"60, Parma Crescent, London, Greater London SW1...",Terraced,2.0,{'imageUrl': 'https://media.rightmove.co.uk/di...,True,"{'lat': 51.4622, 'lng': -0.16433}",https://www.rightmove.co.uk/house-prices/detai...,"£775,000",24 Mar 2023,Freehold,False,775000,Parma Crescent,London,60,SW11 1LU,NaN
5,"60, Parma Crescent, London, Greater London SW1...",Terraced,2.0,{'imageUrl': 'https://media.rightmove.co.uk/di...,True,"{'lat': 51.4622, 'lng': -0.16433}",https://www.rightmove.co.uk/house-prices/detai...,"£620,000",29 Jun 2018,Freehold,False,620000,Parma Crescent,London,60,SW11 1LU,NaN
6,"Flat B, 73, Parma Crescent, London, Greater Lo...",Flat,1.0,{'imageUrl': 'https://media.rightmove.co.uk/di...,True,"{'lat': 51.4622, 'lng': -0.16433}",https://www.rightmove.co.uk/house-prices/detai...,"£775,000",17 Mar 2023,Leasehold,False,775000,Parma Crescent,London,73,SW11 1LU,Flat B
7,"Flat B, 73, Parma Crescent, London, Greater Lo...",Flat,1.0,{'imageUrl': 'https://media.rightmove.co.uk/di...,True,"{'lat': 51.4622, 'lng': -0.16433}",https://www.rightmove.co.uk/house-prices/detai...,"£480,000",3 Sep 2012,Leasehold,False,480000,Parma Crescent,London,73,SW11 1LU,Flat B
8,"Flat B, 73, Parma Crescent, London, Greater Lo...",Flat,1.0,{'imageUrl': 'https://media.rightmove.co.uk/di...,True,"{'lat': 51.4622, 'lng': -0.16433}",https://www.rightmove.co.uk/house-prices/detai...,"£277,500",24 Aug 2005,Leasehold,False,277500,Parma Crescent,London,73,SW11 1LU,Flat B
9,"Flat B, 73, Parma Crescent, London, Greater Lo...",Flat,1.0,{'imageUrl': 'https://media.rightmove.co.uk/di...,True,"{'lat': 51.4622, 'lng': -0.16433}",https://www.rightmove.co.uk/house-prices/detai...,"£169,995",16 Sep 1999,Leasehold,False,169995,Parma Crescent,London,73,SW11 1LU,Flat B
